In [10]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from mediapipe.framework.formats import landmark_pb2
import mediapipe as mp

In [11]:
def draw_landmarks_on_image(rgb_image, detection_result):
  face_landmarks_list = detection_result.face_landmarks
  annotated_image = np.copy(rgb_image)

  # Loop through the detected faces to visualize.
  for idx in range(len(face_landmarks_list)):
    face_landmarks = face_landmarks_list[idx]

    # Draw the face landmarks.
    face_landmarks_proto = landmark_pb2.NormalizedLandmarkList()
    face_landmarks_proto.landmark.extend([
      landmark_pb2.NormalizedLandmark(x=landmark.x, y=landmark.y, z=landmark.z) for landmark in face_landmarks
    ])

    mp.solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_TESSELATION,
        landmark_drawing_spec=None,
        connection_drawing_spec=mp.solutions.drawing_styles
        .get_default_face_mesh_tesselation_style())
    mp.solutions.drawing_utils.draw_landmarks(
        image=annotated_image,
        landmark_list=face_landmarks_proto,
        connections=mp.solutions.face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp.solutions.drawing_styles
          .get_default_face_mesh_iris_connections_style())

  return annotated_image

In [12]:
model_path = "face_landmarker.task"

In [13]:
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode

options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)

In [94]:
cap = cv2.VideoCapture(0)
while True:
    ret, image = cap.read()
    
    if not ret:
        print("Can't receive frame (stream end?). Exiting ...")
        break

    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=image)
    detector = FaceLandmarker.create_from_options(options)
    face_landmarker_result = detector.detect(mp_image)
    annotated_image = draw_landmarks_on_image(image, face_landmarker_result)

    cv2.imshow('frame', annotated_image)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

W20230905 16:54:48.836284 12862371 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230905 16:54:48.927917 12862371 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230905 16:54:49.028632 12862371 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230905 16:54:49.096297 12862371 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230905 16:54:49.156564 12862371 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230905 16:54:49.225361 12862371 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
W20230905 16:54:49.287953 12862371

In [16]:
# считывание фотографии
img = cv2.imread("image.jpg")
cv2_imshow(img)

[ WARN:0@121.887] global loadsave.cpp:248 findDecoder imread_('image.jpg'): can't open/read file: check file path/integrity


NameError: name 'cv2_imshow' is not defined

In [41]:
directory_image_path = "emotion-dataset"

emotion = ["angry", "disgust", "fear", "happy", "neutral", "sad", "surprise"]
emotion_cords_dataset = list()
for dir_emotion in emotion:
    dir_emotion_path = f"{directory_image_path}/{dir_emotion}"
    
    file_names = [file for file in os.listdir(dir_emotion_path) if os.path.isfile(os.path.join(dir_emotion_path, file))]

    for image in file_names:
        img = cv2.imread(f"{dir_emotion_path}/{image}")
        
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=img)
        detector = FaceLandmarker.create_from_options(options)
        face_landmarker_result = detector.detect(mp_image)
        face_landmarker_result = face_landmarker_result.face_landmarks[0]

        cords_image = list(map(lambda landmark_xyz: [landmark_xyz.x, landmark_xyz.y, landmark_xyz.z], face_landmarker_result))
        print(len(cords_image))
        break
    break

[[0.4810728132724762, 0.6631850004196167, -0.06696369498968124], [0.47256869077682495, 0.5614916682243347, -0.14556093513965607], [0.47526755928993225, 0.5941227078437805, -0.0756843313574791], [0.4503993093967438, 0.4572731554508209, -0.11380236595869064], [0.4734130799770355, 0.5295853614807129, -0.1564757078886032], [0.4753653109073639, 0.4891500771045685, -0.1462850421667099], [0.4793171286582947, 0.38579225540161133, -0.07604793459177017], [0.25756120681762695, 0.3713952898979187, 0.026727821677923203], [0.4819544553756714, 0.29327499866485596, -0.05428897216916084], [0.483920156955719, 0.25211504101753235, -0.0576309859752655], [0.48951318860054016, 0.13630181550979614, -0.030775586143136024], [0.4814746677875519, 0.6727100014686584, -0.06195914000272751], [0.481077641248703, 0.6816500425338745, -0.05289062112569809], [0.4797517657279968, 0.6903607845306396, -0.03780526667833328], [0.4799444079399109, 0.6902796030044556, -0.0361512117087841], [0.478851854801178, 0.702114999294281

W20230906 10:08:55.402420 13049060 face_landmarker_graph.cc:168] Face blendshape model contains CPU only ops. Sets FaceBlendshapesGraph acceleration to Xnnpack.
